In [4]:
import numpy as np
from dataclasses import dataclass
from sklearn.datasets import load_diabetes
import pandas as pd

/var/folders/1j/chhd52wn0z97rq9skbstg0_00000gn/T/ipykernel_3418/2344207688.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
@dataclass
class SplitData:
    """
    A dataclass for representing the result of a data split at a Decision Tree node, including details necessary for propagating splits across the tree.

    Attributes:
        left_partition_mask (np.ndarray): A boolean mask indicating samples allocated to the left child node based on the split.
        right_partition_mask (np.ndarray): A boolean mask indicating samples allocated to the right child node based on the split.
        left_count (int): Number of samples in the left partition, facilitating efficient calculation of statistics or further splits.
        right_count (int): Number of samples in the right partition, facilitating efficient calculation of statistics or further splits.
        feat_index (int): Index of the feature used for splitting, enabling traceability of the decision path within the tree.
        cost (float): The cost or impurity reduction achieved by this split, used to assess the quality of the split.
        thresh (float): Threshold value for the feature at this split, determining the boundary between left and right partitions.
        
    This structure is crucial for the iterative construction and analysis of Decision Trees, where each node's decision to split data affects subsequent nodes' decisions.
    """
    left_partition_mask: np.ndarray
    right_partition_mask: np.ndarray
    left_count: int
    right_count: int 
    feat_index: int 
    cost: float 
    thresh: float

    def __str__(self):
        cost_rounded = float(self.cost)
        cost_rounded = round(cost_rounded, 3)

        thresh_rounded = float(self.thresh)
        thresh_rounded = round(thresh_rounded, 3)

        return ', '.join([
            f'cost={cost_rounded}', 
            f'feature_index={self.feat_index}', 
            f'left_count={self.left_count}',
            f'right_count={self.right_count}', 
            f'thresh={thresh_rounded}',
        ])
    

class TreeNode:
    def __init__(self, 
                    X : np.array, 
                    y : np.array,
                    max_depth=5, 
                    min_leaves=1,
                    depth=0,
                    node_type='init_node',
                    partition=None,
                    
                 ):
        
        self.depth = depth
        self.max_depth = max_depth 
        self.min_leaves = min_leaves

        split_data = self.split_by_min_feat_thresh(X, y, partition)
        
        next_depth = self.depth + 1
        is_not_max_depth = self.max_depth > next_depth        
        left_is_not_max_leaves = min_leaves < split_data.left_count
        right_is_not_max_leaves = min_leaves < split_data.right_count
        
        grow_left = is_not_max_depth and left_is_not_max_leaves
        grow_right = is_not_max_depth and right_is_not_max_leaves

        self.feat_index = split_data.feat_index
        self.thresh = split_data.thresh 

        print(f'[{self.depth}/{self.max_depth}] node_type={node_type}, grow_left={grow_left}, grow_right={grow_right}, {split_data}')

        self.left_node, self.right_node = None, None
        
        if grow_left:
            self.left_node = TreeNode(
                                X, y, 
                                max_depth=max_depth, 
                                min_leaves=min_leaves,
                                depth=next_depth,
                                node_type='left_node',
                                partition=split_data.left_partition_mask
                            )

        if grow_right:
            self.right_node = TreeNode(
                                X,y,
                                max_depth=max_depth,
                                min_leaves=min_leaves,
                                depth=next_depth,
                                node_type='right_node',
                                partition=split_data.right_partition_mask
                            )
    
    def split_by_min_thresh(self, X, y, feat_index, partition_mask=None):
        thresh = np.unique(X[:,feat_index]).reshape(1,-1)
        selected_feat = X[:,feat_index].reshape(-1,1)
        is_left_sampler = (selected_feat <= thresh)
        is_right_sampler = ~is_left_sampler

        if partition_mask is not None:
            partition_mask = np.hstack(is_left_sampler.shape[1] * [ partition_mask.reshape(-1,1) ])
            is_left_sampler = is_left_sampler & partition_mask
            is_right_sampler = is_right_sampler & partition_mask

        stacked_targets = np.hstack([y.reshape(-1,1)] * is_left_sampler.shape[1])
        left_sampled, right_sampled = is_left_sampler * stacked_targets, is_right_sampler * stacked_targets

        n_samples_left, n_samples_right = np.sum(is_left_sampler, axis=0), np.sum(is_right_sampler, axis=0)

        with np.errstate(divide='ignore', invalid='ignore'):
            left_samples_mean = np.where(n_samples_left == 0, 0, np.sum(left_sampled, axis=0) / n_samples_left)
            right_samples_mean = np.where(n_samples_right == 0, 0, np.sum(right_sampled, axis=0) / n_samples_right)
        
        var_left = is_left_sampler * (left_sampled - left_samples_mean.reshape(1,-1)) ** 2
        var_right = is_right_sampler * (right_sampled - right_samples_mean.reshape(1,-1)) ** 2

        with np.errstate(divide='ignore', invalid='ignore'):
            var_left = np.where(n_samples_left == 0, 0, np.sum(var_left, axis=0) / n_samples_left)
            var_right = np.where(n_samples_right == 0, 0, np.sum(var_right, axis=0) / n_samples_right)

        total_samples = left_sampled.shape[0]
        assert total_samples == right_sampled.shape[0], 'wrong total samples'

        ratio_left, ratio_right = n_samples_left / total_samples, n_samples_right / total_samples
        thresh_cost = ratio_left * var_left + ratio_right * var_right
        min_thresh_index = np.argmin(thresh_cost)

        return SplitData(
            left_partition_mask=is_left_sampler[:, min_thresh_index],
            right_partition_mask=is_right_sampler[:, min_thresh_index],
            left_count=np.sum(is_left_sampler[:, min_thresh_index]),
            right_count=np.sum(is_right_sampler[:, min_thresh_index]),
            feat_index=feat_index,
            cost=np.min(thresh_cost),
            thresh=thresh.reshape(-1)[min_thresh_index] 
        )

    def split_by_min_feat_thresh(self, X, y, partition):
        feat_count = X.shape[1]
        min_split_data = self.split_by_min_thresh(X, y, 0)

        for feat_index in range(1, feat_count):
            split_data = self.split_by_min_thresh(X, y, feat_index, partition)
            if min_split_data.cost > split_data.cost:
                min_split_data = split_data

        return min_split_data


In [ ]:
data = load_diabetes()
X,y = data['data'], data['target']

max_depth = 20
node = TreeNode(X,y, max_depth=20)

In [7]:

from sklearn.datasets import load_diabetes
import pandas as pd

class DecisionTreeRegressor:
    def __init__(
            self,
            max_depth=5, 
            min_leaves=1, 
        ):

        self.max_depth=max_depth
        self.min_leaves=min_leaves


    def fit(self, X, y):
        self.node = TreeNode(X, y, 
                             max_depth=self.max_depth, 
                             min_leaves=self.min_leaves, 
                             depth=0
                            )
        


In [8]:
data = load_diabetes()
X,y = data['data'], data['target']
max_depth = 10
min_leaves = 2

model = DecisionTreeRegressor(max_depth=10, min_leaves=2)
model.fit(X,y)

[0/10] node_type=init_node, grow_left=True, grow_right=True, cost=4201.076, feature_index=8, left_count=218, right_count=224, thresh=-0.004
[1/10] node_type=left_node, grow_left=True, grow_right=True, cost=1262.777, feature_index=2, left_count=171, right_count=47, thresh=0.006
[2/10] node_type=left_node, grow_left=True, grow_right=True, cost=766.899, feature_index=6, left_count=87, right_count=84, thresh=0.019
[3/10] node_type=left_node, grow_left=True, grow_right=False, cost=480.771, feature_index=4, left_count=85, right_count=2, thresh=0.049
[4/10] node_type=left_node, grow_left=True, grow_right=True, cost=451.92, feature_index=1, left_count=39, right_count=46, thresh=-0.045
[5/10] node_type=left_node, grow_left=True, grow_right=True, cost=225.88, feature_index=3, left_count=21, right_count=18, thresh=-0.037
[6/10] node_type=left_node, grow_left=True, grow_right=True, cost=89.804, feature_index=8, left_count=13, right_count=8, thresh=-0.036
[7/10] node_type=left_node, grow_left=True,